In [ ]:
from feat import Detector
detector = Detector()

In [ ]:
import os
import pathlib
import cv2
import numpy as np
from feat.plotting import imshow

# Define the resolution and frame rate
resolution = (640, 480)
fps = 12
#micro 40ms and 500ms so between 1 and 12
# Initialize the camera capture object
cap = cv2.VideoCapture(0)

# Set the camera resolution and frame rate
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resolution[0])
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resolution[1])
cap.set(cv2.CAP_PROP_FPS, fps)

# Initialize the array to store the frames
frames = []
arr = [] 
arr2= []
# Get the desktop path
desktop_path = str(pathlib.Path.home() / "Desktop")

# Create the folder to store the images
folder_path = os.path.join(desktop_path, "images")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Initialize the frame counter
frame_count = 0

# Loop over the frames
while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    
    # Check if the frame was successfully captured
    if not ret:
        break

    # Save the frame as an image in the folder
    image_path = os.path.join(folder_path, f"frame_{frame_count}.jpg")
    display = os.path.join(folder_path, f"frame_{0}.jpg")
    cv2.imwrite(image_path, frame)
    
    # Increment the frame counter
    frame_count += 1
    
    # Add the frame to the array
    frames.append(frame)
    
    # Display the frame (optional)
    cv2.imshow('Frame', frame)
    
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera capture object and close the display window
cap.release()
cv2.destroyAllWindows()

# Convert the array to a numpy array and print its shape
frames = np.array(frames)
print(frames.shape)
print(frame_count)
#imshow(display)

i = 0
while i < 4:
    display2 = os.path.join(folder_path, f"frame_{i}.jpg")
    single_face_prediction = detector.detect_image(display2)
    max_col= single_face_prediction[['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']].idxmax(axis=1)
    max_val=single_face_prediction[['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']].max(axis=1)
    #print(max_val)
    #print(max_col)
    arr.append(max_val)
    arr2.append(max_col)
    imshow(display2)
    i=i+1


#single_face_prediction = detector.detect_image(display)
#single_face_prediction
#max_col= single_face_prediction[['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']].idxmax(axis=1)
#max_val=single_face_prediction[['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']].max(axis=1)
#print(max_val)
#print(max_col)
print(arr)

In [ ]:
import pandas as pd
df = pd.DataFrame(arr2)
df.rename(columns = {0:'emotion'}, inplace = True)
df

In [ ]:
#test code for detecting micro expressions
emotions = ["happy", "happy", "happy", "angry", "angry", "happy", "happy", "calm", "calm", "calm", "calm", "calm", "happy", "happy", "happy", "happy", "happy", "happy"]

subarrays = []
current_emotion = None
current_subarray = []
start_index = 0

for i, emotion in enumerate(emotions):
    if current_emotion == emotion:
        current_subarray.append(emotion)
    else:
        if current_emotion is not None and 5 <= len(current_subarray) <= 12:
            subarrays.append((start_index, i-1))
        current_emotion = emotion
        current_subarray = [emotion]
        start_index = i

if current_emotion is not None and 5 <= len(current_subarray) <= 12:
    subarrays.append((start_index, len(emotions)-1))

result = [(start, end) for start, end in subarrays if emotions[start:end+1].count(emotions[start]) == len(emotions[start:end+1]) and len(emotions[start:end+1]) >= 5]

print(result)